# Generate social media posts

Setting up sap version of LangChain (sap-llm-commons). Instructions are adapted from https://github.tools.sap/AILearning/genAIDay. To follow these instructions, VPM connection is required. **Need to ask what is at the moment the reccomendation for non-sap employees**

In [ ]:
#!pip install "sap-llm-commons===0.1.0" --extra-index-url https://int.repositories.cloud.sap/artifactory/api/pypi/proxy-deploy-releases-hyperspace-pypi/simple --trusted-host int.repositories.cloud.sap
#!pip install langchain==0.0.281 openai==0.28.1 panel tiktoken pypdf chromadb pycryptodome lark wikipedia DateTime jupyter_bokeh langchain docarray django==2.2 django-cors-headers==2.5.3 django-storages-redux==1.3.3 selenium==4.8.2 panel

In [ ]:
!pip show sap_llm_commons

In [ ]:
#!pip install ai-api-client-sdk
#!pip install ai-core-sdk

After downloading llm-commons you need to create a folder named .aicore_llm in your home directory and inside you need to create a config.json file like this:

```
{
    "AICORE_LLM_AUTH_URL": " ... ",
    "AICORE_LLM_CLIENT_ID": " ... ",
    "AICORE_LLM_CLIENT_SECRET": " ... ",
    "AICORE_LLM_API_BASE": " ... ",
    "AICORE_LLM_RESOURCE_GROUP": " ... "
}
```
You can create automatically the file by downloading the access key of your AI Core instance and typing in the command line: 
```
aicore-llm configure -k your-aicore-access-key.txt
```

### Let's test llm_commons and langchain

In [7]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(  proxy_model_name='gpt-4' ,\
                    temperature=0.8,\
                    verbose = True)
chat.predict('What is your name?')

"I am an AI language model created by OpenAI, and I don't have a personal name. You can simply refer to me as OpenAI. How can I assist you today?"

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)


## Build the prompt template with langchain

In [1]:
problem_category={'public cleanliness':['a dirty public area','an overflowing dustbin','a piece of bulky waste in a common area','a graffity in a public wall'],
                  'roads or footpaths':['a covered signboard which is not visible from the street','a malfunctioning streetlight','a pot hole in a street','a huge crack in the street'], 
                  'facility or park maintenance':['a fallen tree','some overgrown grass','a malfunctioning light in a park'],
                  'pests':['a bee nest in a public area','a potential mosquito breeding site'],
                  'drains & sewers': [' a chocked drain','an overflowing drain','a damaged drain','a bad sewage smell','a flooding']} 

urgency_description= { 'LOW': 'not a urgent situation. It does not pose any concern with public safety and does not require immediate attention.', 
    'MEDIUM': ' mildly urgent. It does not cause any immediate danger, but it impacts negatively and significantly the life of people in the neighborhood.',
    'HIGH': ' urgent. It needs to be resolved quickly because it can potentially cause dangerous situations or disruptions.',
    'VERY HIGH': 'extremely urgent. the issue needs to be handled a immediately, as it is a matter of public safety.'
}
tone= ['polite','sarcastic','rude, almost offensive',]


In [2]:
template_string = """ Generate a social media post written by a citizen \
who wants to report a problem in his neighbourhood. \

Solve the assignment in multiple steps:

1. Invent the situation the citizen wants to report.  The problem has to do with {category} , \
    in particular {problem}. The issue must be {urgency}\

2. Invent the address where this situation has happened. The addreess should be \
    somewhere in Leytonstone (UK). 

3. Generate the social media post of about 100 words where the citizen reports the issue,\
    hoping that the public administration will solve this. \
    The style of the post should be {tone}. \
    The post should mention the address (street, park etc. ) where the situation happened. Use the address you generated in step 2.\
        
Return the social media post only. 

"""

In [3]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [4]:
post = prompt_template.format_messages(
                    category='drains & sewers',
                    problem=problem_category['drains & sewers'][-1],
                    urgency=urgency_description['VERY HIGH'],
                    tone=tone[0])

In [8]:
response=chat(post)

In [9]:
response.content

"🚨 URGENT FLOODING ALERT 🚨\n\nDear fellow residents of Leytonstone,\n\nI hope this post finds you well. I am writing to bring your immediate attention to a critical problem in our neighborhood that demands urgent action from the local authorities. The drains and sewers in our area, specifically on High Road near Leytonstone Park, are severely flooded, posing a significant risk to public safety.\n\nThe current situation is dire, and we urgently need the support of the public administration to rectify this issue as soon as possible. Our community's safety is at stake, and it is crucial that the necessary measures are taken promptly to prevent further damage and potential hazards.\n\nI kindly request all residents to share this message widely, ensuring that it reaches the concerned authorities who can swiftly address this alarming situation. Together, let's raise our voices and advocate for the immediate resolution of this flooding problem on High Road near Leytonstone Park.\n\nThank you 

## Generate a dataframe to collect the prompts

In [10]:
import pandas as pd
import numpy as np

df=pd.DataFrame(data={'citizen':['-' for i in range(0,100)],
                      'tone':['-' for i in range(0,100)],
                      'problem_category':['-' for i in range(0,100)],
                      'problem':['-' for i in range(0,100)],
                      'urgency':['-' for i in range(0,100)],
                      'post':['-' for i in range(0,100)],
    })
df

,citizen,tone,problem_category,problem,urgency,post
0,-,-,-,-,-,-
1,-,-,-,-,-,-
2,-,-,-,-,-,-
3,-,-,-,-,-,-
4,-,-,-,-,-,-
...,...,...,...,...,...,...
95,-,-,-,-,-,-
96,-,-,-,-,-,-
97,-,-,-,-,-,-
98,-,-,-,-,-,-


Distribute the posts equally between the team members

In [11]:
citizens=['Alice','Cesare','Jacob','Mostafa','Trinidad']
df['citizen']=np.random.choice(citizens, size=len(df))

df

,citizen,tone,problem_category,problem,urgency,post
0,Mostafa,-,-,-,-,-
1,Mostafa,-,-,-,-,-
2,Jacob,-,-,-,-,-
3,Cesare,-,-,-,-,-
4,Jacob,-,-,-,-,-
...,...,...,...,...,...,...
95,Alice,-,-,-,-,-
96,Trinidad,-,-,-,-,-
97,Mostafa,-,-,-,-,-
98,Mostafa,-,-,-,-,-


Assign the tone according to certain probabilities

In [12]:
tone= ['polite','a bit ironic','angry and sarcastic','rude, excessive']
probabilities=[0.55,0.25,0.15,0.05]
df['tone']=np.random.choice(tone, size=len(df), p=probabilities)
df

,citizen,tone,problem_category,problem,urgency,post
0,Mostafa,polite,-,-,-,-
1,Mostafa,"rude, excessive",-,-,-,-
2,Jacob,polite,-,-,-,-
3,Cesare,polite,-,-,-,-
4,Jacob,polite,-,-,-,-
...,...,...,...,...,...,...
95,Alice,polite,-,-,-,-
96,Trinidad,angry and sarcastic,-,-,-,-
97,Mostafa,a bit ironic,-,-,-,-
98,Mostafa,polite,-,-,-,-


Now assign the problem category and the problem

In [13]:
categories=['public cleanliness','roads or footpaths','facility or park maintenance','pests','drains & sewers']
probabilities=[0.35,0.25,0.25,0.05,0.10]
df['problem_category']=np.random.choice(categories, size=len(df), p=probabilities)


In [14]:
problems=[]
for index, row in df.iterrows():
    problems.append(np.random.choice(problem_category[row['problem_category']], size=1)[0])
df['problem']=problems
df

,citizen,tone,problem_category,problem,urgency,post
0,Mostafa,polite,public cleanliness,a dirty public area,-,-
1,Mostafa,"rude, excessive",public cleanliness,an overflowing dustbin,-,-
2,Jacob,polite,public cleanliness,a graffity in a public wall,-,-
3,Cesare,polite,public cleanliness,a dirty public area,-,-
4,Jacob,polite,drains & sewers,a bad sewage smell,-,-
...,...,...,...,...,...,...
95,Alice,polite,roads or footpaths,a huge crack in the street,-,-
96,Trinidad,angry and sarcastic,roads or footpaths,a covered signboard which is not visible from ...,-,-
97,Mostafa,a bit ironic,public cleanliness,a graffity in a public wall,-,-
98,Mostafa,polite,public cleanliness,a graffity in a public wall,-,-


Now the urgency

In [15]:
df['urgency']=np.random.choice(['LOW','MEDIUM'], size=len(df))
urgencies=[]
for index, row in df.iterrows():
    if row['problem']  in ['a huge crack in the street', 'a flooding','a bad sewage smell','a fallen tree']:
        urgencies.append( np.random.choice(['HIGH','VERY HIGH'], size=1)[0] )
    else :
        urgencies.append( row['urgency'])
                    
df['urgency']=urgencies

In [16]:
np.unique(df.urgency)

array(['HIGH', 'LOW', 'MEDIUM', 'VERY HIGH'], dtype=object)

## Generate the posts

Sleep 2 seconds between generating 2 posts, to avoid reaching the max number of call limit

In [17]:
import time 
posts=[]
for index, row in df.iterrows():
    post = prompt_template.format_messages(
                    category=row['problem_category'],
                    problem=row['problem'],
                    urgency=urgency_description[row['urgency']],
                    tone=row['tone']
                    )
    response=chat(post) 
    posts.append(response.content)
    time.sleep(2)
df['post']=posts

In [22]:
df

,citizen,tone,problem_category,problem,urgency,post
0,Mostafa,polite,public cleanliness,a dirty public area,MEDIUM,"""📢 Attention to all Sagenai residents! 🏡🌳\n\nI..."
1,Mostafa,"rude, excessive",public cleanliness,an overflowing dustbin,MEDIUM,🚨🗑️ATTENTION NEIGHBORS OF LEYTONSTONE🗑️🚨\n\nI ...
2,Jacob,polite,public cleanliness,a graffity in a public wall,LOW,📢 Attention Sagenai residents! 📢\n\n🏙️ I would...
3,Cesare,polite,public cleanliness,a dirty public area,MEDIUM,📢 Attention Sagenai residents! 📢\n\n🌳 I hope t...
4,Jacob,polite,drains & sewers,a bad sewage smell,VERY HIGH,📢 Urgent Public Announcement 📢\n\nDear fellow ...
...,...,...,...,...,...,...
95,Alice,polite,roads or footpaths,a huge crack in the street,VERY HIGH,📢 Attention Sagenai residents! 📢\n\n🚨 Urgent P...
96,Trinidad,angry and sarcastic,roads or footpaths,a covered signboard which is not visible from ...,MEDIUM,🚧🚨 ATTENTION RESIDENTS OF LEYTONSTONE 🚨🚧\n\nCa...
97,Mostafa,a bit ironic,public cleanliness,a graffity in a public wall,MEDIUM,🚨 Urgent! Attention citizens of Sagenai 🚨\n\n👋...
98,Mostafa,polite,public cleanliness,a graffity in a public wall,MEDIUM,"""📢 Attention, fellow neighbors of Sagenai! 📢\n..."


In [ ]:
#df.to_excel('posts3.xlsx')

## Add some non-relevant posts


In [23]:
trolls=[]
reasons=['complain about public administration',
         'state how this community is useless',
         'complain about 4G'
         ]
template_troll=''' You are a troll and you want to make some noise on a social media page \
    that citizens uses to report issues in their neighborhood. \
    Generate a social media post to {reason}.
'''
prompt_template_troll = ChatPromptTemplate.from_template(template_troll)

for i in range(10):
    troll_post = prompt_template_troll.format_messages(
                    reason=np.random.choice(reasons,size=1)[0])
    troll = chat(troll_post)
    trolls.append(troll.content)

df_troll=pd.DataFrame()

In [ ]:
df_troll=pd.DataFrame(data={ 'citizen':np.random.choice(citizens, size=len(trolls)),
                            'tone':['spam' for i in range(len(trolls))],
                            'problem_category':['spam' for i in range(len(trolls))],
                            'problem':['spam' for i in range(len(trolls))],
                            'urgency':['spam' for i in range(len(trolls))],
                            'post':trolls,
    })
df_troll

In [ ]:
df=pd.concat([df,df_troll])
df=df.sample(len(df))
#df.to_excel('posts3.xlsx')

### Add coordinates, replace Leytonstone with Sagenai

In [19]:
def add_coordinates(text_in):
    
    lat_min = 51.544448
    lat_max = 51.577362
    lon_min = -0.033311 
    lon_max = 0.019450
    
    text_out= text_in.replace('Leytonstone','Sagenai').replace('London',' ')+ ''' 
 

        Coordinates:({},{})'''.format(np.random.uniform(lat_min, lat_max), np.random.uniform(lon_min, lon_max))
    return (text_out)

In [20]:
df['post']=df.post.apply( lambda x: add_coordinates(x))
df['post'][5]

"📢 Attention Sagenai residents! 🚧\nYet another example of our wonderful local council's incompetence! 🙄 Just when you thought our roads and footpaths couldn't get any worse, behold the malfunctioning streetlight at the corner of High Street and Grove Park. 🌃💡\nWho needs proper lighting when you can stumble around in the dark, right? 😒 It's a real treat for all the late-night walkers and joggers in the area! 🏃\u200d♂️🚶\u200d♀️\nNo need to worry though, it's not urgent at all! Public safety is clearly not a priority in Sagenai anymore. 😡\nCome on, council, we pay our taxes! Fix this ridiculous problem ASAP! 🛠️\n#IncompetentCouncil #SagenaiRoads #FixTheStreetlight \n \n\n        Coordinates:(51.55463680692228,0.014969679837567224)"

In [26]:
df['post']=df.post.apply( lambda x: x.replace('LEYTONSTONE','SAGENAI'))

In [27]:
df.to_excel('posts6.xlsx', index=False)